## Minimal three node network
Here, we are going to create a network with three nodes, three lines and one generator. We then solve the non-linear power flow using a Newton-Raphson.

In [1]:
import pypsa
import numpy as np

In [3]:
network = pypsa.Network()

# Add three buses
n_buses = 3

for i in range(n_buses):
    network.add("Bus","My bus {}".format(i),
                v_nom=20.)

network.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
My bus 0,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 1,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 2,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,


In [5]:
# Add three lines in a ring
for i in range(n_buses):
    network.add("Line", "My line {}".format(i),
                bus0="My bus {}".format(i),
                bus1="My bus {}".format((i+1)%n_buses),
                x=0.1,
                r=0.01)

network.lines

attribute,bus0,bus1,type,x,r,g,b,s_nom,s_nom_extendable,s_nom_min,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
My line 0,My bus 0,My bus 1,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
My line 1,My bus 1,My bus 2,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
My line 2,My bus 2,My bus 0,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Add a generator at bus 0
network.add("Generator","My gen",
            bus="My bus 0",
            p_set=100,
            control="PQ")

network.generators

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
My gen,My bus 0,PQ,,0.0,False,0.0,inf,0.0,1.0,100.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [7]:
# Add a load at bus 1
network.add("Load","My load",
            bus="My bus 1",
            p_set=100)

network.loads

attribute,bus,carrier,type,p_set,q_set,sign
My load,My bus 1,,,100.0,0.0,-1.0


In [8]:
network.loads.p_set

My load    100.0
Name: p_set, dtype: float64

In [9]:
# Fix the reactive power of the load
network.loads.q_set = 100.

In [11]:
# Do a Newton-Raphson power flow
network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.021002 seconds


{'n_iter':      0
 now  3,
 'error':                 0
 now  4.753531e-10,
 'converged':         0
 now  True}

In [12]:
# Active Power flow
network.lines_t.p0

,My line 0,My line 1,My line 2
now,66.897487,-33.333333,-33.391038


In [13]:
# Voltage angles on the buses
network.buses_t.v_ang*180/np.pi

,My bus 0,My bus 1,My bus 2
now,0.0,-0.875939,-0.433813


In [14]:
# Voltage magnitude on the buses
network.buses_t.v_mag_pu

,My bus 0,My bus 1,My bus 2
now,1.0,0.981199,0.99057
